# MICOM medium for VMH fermented foods

Here we will build up the environmental conditions used for modeling. We will start from the metabolite abundances obtained from the VMH diet designer and basically go through the following step.

1. Convert to fluxes and adjust very low abundant compounds.
2. Dilute metabolites absorbed in the small intestine.
3. Add primary bile acids and mucin cores.
4. Add in missing components to allow for at least slow growth for all known taxa residing in the human gut.

That should leave us with a set of usable media for all later simulation steps.

But first let us inspect the actual diet data we got. For that we will read the diet data, rearrange it a bit and add in annotations.

In [1]:
import pandas as pd


eu = pd.read_csv("../data/vmh_high_fiber.tsv", sep="\t").iloc[:, 0:2]
eu.columns = ["reaction", "flux"]
eu.flux *= 0.8

diet = pd.read_csv("../data/vmh_fermented.tsv", sep="\t", header=None)
diet.columns = ["reaction", "flux"]

diet = pd.concat([eu, diet])
diet.reaction = diet.reaction.str.replace("\\[e\\]$", "(e)", regex=True)
diet = diet.groupby("reaction").flux.sum().reset_index()
annotations = pd.read_csv("../data/agora_metabolites.csv")

diet = diet.rename(columns={diet.columns[0]: "reaction"})
diet["metabolite"] = diet.reaction.str.replace("^EX_", "", regex=True).str.replace("\\[e\\]|\\(e\\)", "", regex=True)
diet.loc[diet.metabolite == "4hpro", "metabolite"] = "4hpro_LT"  # fix name for hydroxyproline
diet.loc[diet.flux == 0, "flux"] = 1e-4  # bug in VMH designer where everything <1e-4 gets truncated to 0

diet

,reaction,flux,metabolite
0,EX_10fthf(e),0.000204,10fthf
1,EX_5mthf(e),0.000302,5mthf
2,EX_CE2510(e),0.874191,CE2510
3,EX_CE4843(e),0.030427,CE4843
4,EX_adpcbl(e),0.000004,adpcbl
...,...,...,...
91,EX_urate(e),5.055348,urate
92,EX_val_L(e),45.648629,val_L
93,EX_vitd3(e),0.000019,vitd3
94,EX_xylt(e),0.220841,xylt


## Adjust for intestinal adsorption

To achieve this we will load the Recon3 human model. AGORA and Recon IDs are very similar so we should be able to match them. We just have to adjust the Recon3 ones a bit. We start by identifying all available exchanges in Recon3 and adjusting the IDs.

In [2]:
from cobra.io import read_sbml_model
import pandas as pd

recon3 = read_sbml_model("../data/Recon3D.xml.gz")
exchanges = pd.Series([r.id for r in recon3.exchanges])
exchanges = exchanges.str.replace("__", "_").str.replace("_e$|EX_", "", regex=True)
exchanges.head()

0     5adtststerone
1    5adtststerones
2             5fthf
3             5htrp
4             5mthf
dtype: object

In [3]:
diet["dilution"] = 1.0
diet.loc[diet.metabolite.isin(exchanges), "dilution"] = 0.2
diet["flux"] = diet["flux"] * diet["dilution"] 
diet[["metabolite", "dilution"]].drop_duplicates().dilution.value_counts()

0.2    83
1.0    13
Name: dilution, dtype: int64

## Adding host supplied components

Finally we add the host metabolites such as primary bile acids and mucins and a minuscule amount of oxygen.

In [4]:
diet.set_index("metabolite", inplace=True)

# mucin
for met in annotations.loc[annotations.metabolite.str.contains("core"), "metabolite"]:
    diet.loc[met, "flux"] = 1

# primary BAs
for met in ["gchola", "tchola"]:
    diet.loc[met, "flux"] = 1

# fiber
diet.loc["cellul", "flux"] = 0.1

# anaerobic
diet.loc["o2", "flux"] = 0.001

diet.reset_index(inplace=True)
diet["reaction"] = "EX_" + diet.metabolite + "(e)"
diet

,metabolite,reaction,flux,dilution
0,10fthf,EX_10fthf(e),0.000041,0.2
1,5mthf,EX_5mthf(e),0.000060,0.2
2,CE2510,EX_CE2510(e),0.174838,0.2
3,CE4843,EX_CE4843(e),0.030427,1.0
4,adpcbl,EX_adpcbl(e),0.000004,1.0
...,...,...,...,...
107,gncore2_rl,EX_gncore2_rl(e),1.000000,NaN
108,core7,EX_core7(e),1.000000,NaN
109,gchola,EX_gchola(e),1.000000,NaN
110,tchola,EX_tchola(e),1.000000,NaN


And we will merge this tbale with some annotations to make it more accessible.

In [5]:
skeleton = pd.merge(diet, annotations, on="metabolite")

skeleton["global_id"] = skeleton.reaction
skeleton["reaction"] = "EX_" + skeleton.metabolite + "_m"
skeleton.head()

,metabolite,reaction,flux,dilution,name,hmdb,kegg.compound,pubchem.compound,inchi,chebi,global_id
0,10fthf,EX_10fthf_m,0.000041,0.2,10-Formyltetrahydrofolate,HMDB00972,C00234,122347.0,NaN,NaN,EX_10fthf(e)
1,5mthf,EX_5mthf_m,0.000060,0.2,5-Methyltetrahydrofolate,HMDB01396,C00440,439234.0,NaN,NaN,EX_5mthf(e)
2,ala_D,EX_ala_D_m,0.345144,0.2,D-alanine,HMDB01310,C00133,71080.0,"InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5...",NaN,EX_ala_D(e)
3,ala_L,EX_ala_L_m,9.794952,0.2,L-alanine,HMDB00161,C00041,5950.0,"InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5...",NaN,EX_ala_L(e)
4,arach,EX_arach_m,0.112914,0.2,arachidate,HMDB02212,C06425,10467.0,InChI=1S/C20H40O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,NaN,EX_arach(e)


## Complete the medium

Great we now have a pretty good skeleton. One issue that this will never be fully complete. There will always be some components missing that are essential for microbial growth. Fortunately, we provide a algorithm in MICOM to complete a medium with the smallest set of additional components to provide growth to all intestinal taxa.

In [6]:
from micom.workflows.db_media import complete_db_medium

manifest, imports = complete_db_medium("../data/agora103_strain.qza", skeleton, growth=0.05, threads=12, max_added_import=10, weights="mass")

Output()

In [7]:
manifest.can_grow.value_counts()

True     527
False    291
Name: can_grow, dtype: int64

In [8]:
filled = imports.max()
added = filled[~filled.index.isin(skeleton.reaction)]

print(f"Added flux is {added.sum():.2f}/{filled.sum():.2f} mmol/h.")

Added flux is 11.17/38403.83 mmol/h.


Let's see what was added in large amounts.

In [9]:
added.sort_values(ascending=False)[0:20]

EX_h2_m         5.819827
EX_h_m          4.331678
EX_for_m        0.247297
EX_h2s_m        0.192050
EX_asn_L_m      0.065847
EX_xyl_D_m      0.059302
EX_ura_m        0.056677
EX_xan_m        0.054624
EX_rib_D_m      0.040283
EX_gln_L_m      0.034262
EX_nmn_m        0.030450
EX_co2_m        0.028872
EX_hxan_m       0.023320
EX_ade_m        0.022087
EX_arabttr_m    0.020157
EX_3mop_m       0.017679
EX_2obut_m      0.016349
EX_csn_m        0.014682
EX_adn_m        0.011045
EX_dad_2_m      0.011045
dtype: float64

Looks okay. So we will now assemble the final medium. For this we add the new components to each sample and rebuild the annotations for a nicely formatted medium.

In [10]:
added_df = added.reset_index() 
added_df.iloc[:, 0] = added_df.iloc[:, 0].str.replace("EX_|_m$", "", regex=True)
added_df.columns = ["metabolite", "flux"]
added_df = pd.concat([skeleton[["metabolite", "flux"]], added_df])

completed = pd.merge(added_df, annotations, on="metabolite", how="left")
completed["reaction"] = "EX_" + completed.metabolite + "_m"
completed["global_id"] = "EX_" + completed.metabolite + "(e)"
completed

,metabolite,flux,name,hmdb,kegg.compound,pubchem.compound,inchi,chebi,reaction,global_id
0,10fthf,0.000041,10-Formyltetrahydrofolate,HMDB00972,C00234,122347.0,NaN,NaN,EX_10fthf_m,EX_10fthf(e)
1,5mthf,0.000060,5-Methyltetrahydrofolate,HMDB01396,C00440,439234.0,NaN,NaN,EX_5mthf_m,EX_5mthf(e)
2,ala_D,0.345144,D-alanine,HMDB01310,C00133,71080.0,"InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5...",NaN,EX_ala_D_m,EX_ala_D(e)
3,ala_L,9.794952,L-alanine,HMDB00161,C00041,5950.0,"InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5...",NaN,EX_ala_L_m,EX_ala_L(e)
4,arach,0.112914,arachidate,HMDB02212,C06425,10467.0,InChI=1S/C20H40O2/c1-2-3-4-5-6-7-8-9-10-11-12-...,NaN,EX_arach_m,EX_arach(e)
...,...,...,...,...,...,...,...,...,...,...
141,acmana,0.005002,N-acetyl-D-mannosamine,HMDB01129,C00645,439281.0,InChI=1S/C8H15NO6/c1-4(12)9-5(2-10)7(14)8(15)6...,NaN,EX_acmana_m,EX_acmana(e)
142,MGlcn188,0.004624,mucin-type O-glycan No 188,NaN,NaN,NaN,NaN,NaN,EX_MGlcn188_m,EX_MGlcn188(e)
143,no2,0.001486,Nitrite,HMDB02786,C00088,24529.0,"InChI=1S/HNO2/c2-1-3/h(H,2,3)/p-1",NaN,EX_no2_m,EX_no2(e)
144,xan,0.054624,Xanthine,HMDB00292,C00385,1188.0,InChI=1S/C5H4N4O2/c10-4-2-3(7-1-6-2)8-5(11)9-4...,NaN,EX_xan_m,EX_xan(e)


## Validate the medium

And we will now validate whether the medium works.

In [11]:
from micom.workflows.db_media import check_db_medium

check = check_db_medium("../data/agora103_strain.qza", medium=completed, threads=12)

Output()

In [12]:
check.growth_rate.describe()

count    818.000000
mean       0.077710
std        0.076164
min        0.000000
25%        0.006765
50%        0.050000
75%        0.128204
max        0.323333
Name: growth_rate, dtype: float64

And we are done now and will the save the final medium.

In [13]:
import qiime2 as q2

arti = q2.Artifact.import_data("MicomMedium[Global]", completed)
arti.save("../media/vmh_fermented_agora.qza")

'../media/vmh_fermented_agora.qza'